## Course Project: Text Classification with DBpedia Dataset

### Description:
In this project, you will be working with the DBpedia ontology dataset. The task is to classify textual descriptions into one of 14 classes such as "Company", "Artist", "Athlete", and so forth. This classification problem will allow you to apply and solidify your understanding of NLP and machine learning concepts in a hands-on manner.

### Objective:
- Understand and process a real-world dataset.
- Implement a text classification model using the GPT-2 architecture.
- Evaluate the performance of your model and aim to achieve the highest accuracy possible.

### Tasks:

#### 1. Dataset Exploration 
- Load the `dbpedia_14` dataset.
- Analyze the dataset: understand the distribution of labels, the length of textual descriptions, etc.
- Split the dataset into training and test sets.

#### 2. Data Pre-processing 
- Tokenize the textual descriptions.
- Ensure that your data is in the appropriate format for model training (e.g., tensors).

#### 3. Model Building 
- Define a classification model using the GPT-2 architecture.
- Implement the forward pass.
- Choose an appropriate loss function for multi-class classification.

#### 4. Model Training 
- Implement a training loop.
- Make sure to track the loss over time. This will help you understand if your model is learning.
- If time permits, play around with hyperparameters to see if you can get better results.

#### 5. Model Evaluation
- Evaluate your model on the test dataset.
- Compute classification metrics such as accuracy, F1 score, etc.
- (Optional) Analyze cases where your model fails. What can you infer from these mistakes?

#### 6. Discussion & Reflection 
- Share your results with the class.
- Reflect on what you've learned: challenges faced, insights gained, and potential improvements.


### Deliverables:
1. A Jupyter Notebook or Python script with:
    - Code for each task.
    - Comments explaining your thought process.
    - Results and visualizations (if any).
2. A brief report summarizing your findings, reflecting on your model's performance, and suggesting potential improvements.

In [ ]:
# Import necessary libraries
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2Model
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, f1_score

# Task 1: Dataset Exploration
# Load the dbpedia_14 dataset
dataset = load_dataset('dbpedia_14')

# Quick exploration
print(dataset['train'].shape)
print(dataset['train'].features)
print(dataset['train'][0])

# Task 2: Data Pre-processing
# Tokenize the textual descriptions
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
tokenizer.pad_token = tokenizer.eos_token

def tokenize_and_format(examples):
    encodings = tokenizer(examples['content'], truncation=True, padding='max_length', max_length=256)
    encodings['labels'] = examples['label']
    return encodings

tokenized_datasets = dataset.map(tokenize_and_format, batched=True)
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

train_dataloader = DataLoader(tokenized_datasets['train'], shuffle=True, batch_size=8)
test_dataloader = DataLoader(tokenized_datasets['test'], batch_size=8)

# Task 3: Model Building
class GPT2ForClassification(nn.Module):
    def __init__(self, num_labels=14):
        super(GPT2ForClassification, self).__init__()
        self.gpt2 = GPT2Model.from_pretrained('gpt2-medium')
        self.classifier = nn.Linear(self.gpt2.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask=None):
        outputs = self.gpt2(input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state
        logits = self.classifier(hidden_states[:, -1])
        return logits

model = GPT2ForClassification().to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Task 4: Model Training
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
criterion = nn.CrossEntropyLoss()

num_epochs = 2  # Sample value. Can be increased as needed.
for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        optimizer.zero_grad()
        
        inputs, masks, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
        inputs, masks, labels = inputs.to(device), masks.to(device), labels.to(device)
        
        logits = model(inputs, masks)
        loss = criterion(logits, labels)
        
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1} completed!")

# Task 5: Model Evaluation
model.eval()

all_predictions = []
all_true_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        inputs, masks, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
        inputs, masks, labels = inputs.to(device), masks.to(device), labels.to(device)

        logits = model(inputs, masks)
        _, preds = torch.max(logits, dim=1)
        
        all_predictions.extend(preds.cpu().numpy())
        all_true_labels.extend(labels.cpu().numpy())

# Compute accuracy and F1 score
accuracy = accuracy_score(all_true_labels, all_predictions)
f1 = f1_score(all_true_labels, all_predictions, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

# Task 6: Discussion & Reflection is more about sharing and understanding the results, so no code is provided for that.
